# Bug triage with Deep Learning

In [1]:
import keras

Using TensorFlow backend.


In [2]:
from __future__ import print_function, division

In [3]:
import re
import numpy as np
import pandas as pd

import os
from tqdm import tqdm_notebook as tqdm
import matplotlib.pyplot as plt
import sys
nb_dir = os.path.split(os.getcwd())[0]
if nb_dir not in sys.path:
    sys.path.append(nb_dir)
    
%matplotlib inline

In [4]:
from keras.layers import Conv1D, Input, Add, Activation, Dropout, Embedding, MaxPooling1D, GlobalMaxPool1D, Flatten, Dense, Concatenate, BatchNormalization
from keras.models import Sequential, Model
from keras.regularizers import l2
from keras.initializers import TruncatedNormal
from keras.layers.advanced_activations import LeakyReLU, ELU
from keras import optimizers

In [5]:
from methods.baseline import Baseline

## Auxiliary methods

## Configurações Globais

In [6]:
MAX_SEQUENCE_LENGTH_T = 20 # 40
MAX_SEQUENCE_LENGTH_D = 200 # 200
EMBEDDING_DIM = 300
MAX_NB_WORDS = 2000

### Parse bugs preproprecessed

In [7]:
# Domain to use
DOMAIN = 'eclipse'
# Dataset paths
DIR = 'data/processed/{}'.format(DOMAIN)
DIR_PAIRS = 'data/normalized/{}'.format(DOMAIN)
DATASET = os.path.join('data/normalized/{}'.format(DOMAIN), '{}.csv'.format(DOMAIN))
# Glove embeddings
GLOVE_DIR='data/embed'
# Log keras
LOG_DIR='logs/training'
# Checkpoint keras
FILE_PATH = "checkpoint_baseline_1000epoch_10steps_1024batch({})".format(DOMAIN)
# Save model
SAVE_PATH = 'baseline_1000epoch_10steps_1024batch({})'.format(DOMAIN)

In [8]:
baseline = Baseline(DIR, DATASET, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

#### Loading bug ids in memory

In [9]:
baseline.load_ids(DIR)
len(baseline.bug_ids)

212512

### Dicionário de títulos e descrições

In [10]:
%%time

baseline.load_preprocess()
len(baseline.sentence_dict)


CPU times: user 37 s, sys: 2.07 s, total: 39 s
Wall time: 38.9 s


## Geração de batches

### Generating tiple of batches

In [11]:
%%time
baseline.prepare_dataset()

Reading train data
Reading the test...



Reading test data
CPU times: user 475 ms, sys: 16.5 ms, total: 492 ms
Wall time: 486 ms


In [12]:
baseline.load_bugs()

In [13]:
baseline.bug_set[2521]

{'bug_severity': '3\n',
 'bug_status': '0\n',
 'component': '633\n',
 'creation_ts': '2001-10-10 22:38:00 -0400',
 'delta_ts': '2005-05-10 14:55:51 -0400',
 'description': 'steps minimize all your windows go to any window and select the window menu pick any window notice that it only gets selected and not maximized this happens in country as well notes',
 'description_word': array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,

In [592]:
%%time

batch_size = 512
batch_size_test = 1024

# we want a constant validation group to have a frame of reference for model performance
train_gen = baseline.siam_gen(baseline.train_data, baseline.dup_sets_train, batch_size, 1)
valid_input_sample, valid_input_pos, valid_input_neg, valid_sim = baseline.batch_iterator(baseline.train_data, 
                                                                                          baseline.dup_sets_train, 
                                                                                          batch_size_test, 1)
test_gen = ([valid_input_sample['title'], valid_input_pos['title'], valid_input_neg['title'], 
             valid_input_sample['description'], valid_input_pos['description'], valid_input_neg['description'],
            valid_input_sample['info'], valid_input_pos['info'], valid_input_neg['info']], valid_sim)

CPU times: user 86.6 ms, sys: 4.8 ms, total: 91.4 ms
Wall time: 90.3 ms


In [540]:
valid_input_sample['title'].shape, valid_input_sample['description'].shape, valid_input_sample['info'].shape, valid_sim.shape

((128, 20), (128, 200), (128, 1682), (128,))

### Validar entrada

In [541]:
%%time 

baseline.display_batch(baseline.train_data, baseline.dup_sets_train, 5)

***Title***: subversive bug report id bxlft
***Title***: subversive bug report id bxlft
***Description***: version i svn client org eclipse team svn connector svnkit i svn svnkit http svnkit com r jvm properties java vendor apple inc osgi bundles default start level org osgi supports framework extension true sun management compiler hot spot organization cardinal eclipse p profile organization os name mac os x osgi ws carbon organization cardinal b osgi instance area file users home workspaces workspace mini rt user name home awt native double buffering true eclipse launcher applications eclipse cpp organization mac os eclipse org osgi framework language en user language en org osgi framework processor x osgi syspath applications eclipse cpp plugins sun boot library path system library frameworks java vm framework versions libraries osgi manifest cache applications eclipse cpp configuration org eclipse osgi manifests osgi compatibility bootdelegation true java version org osgi framework

## Pre-trained embeddings

Loading pretrained word vectors

### Glove

In [17]:
%%time

baseline.generating_embed(GLOVE_DIR=GLOVE_DIR, EMBEDDING_DIM=EMBEDDING_DIM, MAX_NB_WORDS=MAX_NB_WORDS)


Total 1917494 word vectors in Glove 42B 300d.
Found 128207 unique tokens.



CPU times: user 2min 13s, sys: 4.65 s, total: 2min 18s
Wall time: 3min 27s


## Towards Accurate Duplicate Bug Retrieval Using Deep Learning Techniques

https://github.com/tqtg/DuplicateBugFinder

### CNN with filter 3,4,5

In [ ]:
import keras
from keras.layers import GlobalMaxPooling1D, GlobalAveragePooling1D

def cnn_model(embeddings, num_words, embedding_dim, max_sequence_length, trainable):

    embedding_layer = Embedding(num_words,
                              embedding_dim,
                              weights=[embeddings],
                              input_length=max_sequence_length,
                              trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length,), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Yoon Kim model (https://arxiv.org/abs/1408.5882)
    # best combination filter (3, 4, 5) e 128 e 256
    convs = []
    filter_sizes = [3, 4, 5]
    n_filters = 64

    for index, filter_size in enumerate(filter_sizes):
        l_conv = Conv1D(filters=n_filters, kernel_size=filter_size, activation='relu')(embedded_sequences)
        l_pool = MaxPooling1D(pool_size=index+1)(l_conv)
        convs.append(l_pool)

    l_merge = Concatenate(axis=1)(convs)

    # add a 1D convnet with global maxpooling, instead of Yoon Kim model
    #conv = Conv1D(filters=n_filters * 3, kernel_size=3, activation='relu')(l_merge)
    layer = GlobalAveragePooling1D()(l_merge)
    
    layer = Dense(100, activation="relu")(layer)

    cnn_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureCNNGenerationModel') # inputs=visible

    return cnn_feature_model

### Bi-LSTM

In [617]:
from keras.layers import Dense, Input, LSTM, GRU, Dropout, Bidirectional, GlobalAveragePooling1D

def lstm_model(embeddings, num_words, embedding_dim, max_sequence_length, trainable):
    number_lstm_units = 50
    rate_drop_lstm = 0
    recurrent_dropout = 0

    embedding_layer = Embedding(num_words,
                          embedding_dim,
                          weights=[embeddings],
                          input_length=max_sequence_length,
                          trainable=trainable)

    sequence_input = Input(shape=(max_sequence_length, ), name='Feature_BugInput')
    embedded_sequences = embedding_layer(sequence_input)

    # Creating LSTM Encoder
    lstm_layer = Bidirectional(LSTM(number_lstm_units, return_sequences=True, 
                                   dropout=rate_drop_lstm, recurrent_dropout=rate_drop_lstm), 
                               merge_mode='ave')

    layer = lstm_layer(embedded_sequences)
    layer = GlobalAveragePooling1D()(layer)
    layer = Dense(100, activation="relu")(layer)

    lstm_feature_model = Model(inputs=[sequence_input], outputs=[layer], name = 'FeatureLstmGenerationModel') # inputs=visible

    return lstm_feature_model

### MLP

In [ ]:
def mlp_model(input_size):
    info_input = Input(shape=(input_size, ), name='Feature_BugInput')
    input_size = 100
    
    layer = Dense(input_size, activation='relu')(info_input)
    
    mlp_feature_model = Model(inputs=[info_input], outputs=[layer], name = 'FeatureMlpGenerationModel')
    
    return mlp_feature_model

### Siamese model

In [619]:
from keras import backend as K
import tensorflow as tf

def l2_normalize(x, axis):
    norm = K.sqrt(K.sum(K.square(x), axis=axis, keepdims=True))
    return K.maximum(x, K.epsilon()) / K.maximum(norm, K.epsilon())
    #return K.sign(x) * K.maximum(K.abs(x), K.epsilon()) / K.maximum(norm, K.epsilon())

def normalize(x):
    return l2_normalize(x, axis=-1)
    
# https://github.com/keras-team/keras/issues/3031
# https://github.com/keras-team/keras/issues/8335
def cosine_distance(inputs):
    x, y = inputs
    #x = l2_normalize(x, axis=-1)
    #y = l2_normalize(y, axis=-1)
    similarity = K.batch_dot(x, y, axes=1)
    #similarity =  K.sum(x * y, axis=-1)
    distance = K.constant(1) - similarity
    # Distance goes from 0 to 2 in theory, but from 0 to 1 if x and y are both
    # positive (which is the case after ReLU activation).
    return K.mean(distance)

def margin_loss(y_true, y_pred):
    margin = K.constant(1.0)
    loss = K.maximum(0.0, margin - y_pred[0] +  y_pred[1])
    return K.mean(loss)

def pos_distance(y_true, y_pred):
    return K.mean(y_pred[0])

def neg_distance(y_true, y_pred):
    return K.mean(y_pred[1])

def stack_tensors(vects):
    return K.stack(vects)
    #return K.stack(K.squeeze(K.stack(vects), axis=1), axis=2) # stack adds a new dim. So squeeze it
    # better method is to use concatenate
    #return K.concatenate(vects, axis=-1)

In [622]:
from keras.layers import concatenate, Add, Lambda, merge
from keras.optimizers import Adam

def siamese_model(lstm_feature_model, cnn_feature_model, mlp_feature_model, sequence_length_info, 
                  sequence_length_t, sequence_length_d):
  
    bug_t_in = Input(shape = (sequence_length_t, ), name = 'title_in')
    bug_t_pos = Input(shape = (sequence_length_t, ), name = 'title_pos')
    bug_t_neg = Input(shape = (sequence_length_t, ), name = 'title_neg')

    bug_d_in = Input(shape = (sequence_length_d, ), name = 'desc_in')
    bug_d_pos = Input(shape = (sequence_length_d, ), name = 'desc_pos')
    bug_d_neg = Input(shape = (sequence_length_d, ), name = 'desc_neg')
    
    bug_i_in = Input(shape = (sequence_length_info, ), name = 'info_in')
    bug_i_pos = Input(shape = (sequence_length_info, ), name = 'info_pos')
    bug_i_neg = Input(shape = (sequence_length_info, ), name = 'info_neg')

    bug_t_in_feat_lstm = lstm_feature_model(bug_t_in)
    bug_t_pos_feat_lstm = lstm_feature_model(bug_t_pos)
    bug_t_neg_feat_lstm = lstm_feature_model(bug_t_neg)

    bug_d_in_feat_cnn = cnn_feature_model(bug_d_in)
    bug_d_pos_feat_cnn = cnn_feature_model(bug_d_pos)
    bug_d_neg_feat_cnn = cnn_feature_model(bug_d_neg)
    
    bug_i_in_feat_mlp = mlp_feature_model(bug_i_in)
    bug_i_pos_feat_mlp = mlp_feature_model(bug_i_pos)
    bug_i_neg_feat_mlp = mlp_feature_model(bug_i_neg)

#     encoded_anchor = Add(name = 'merge_features_in')([bug_i_in_feat_mlp, bug_t_in_feat_lstm, bug_d_in_feat_cnn])
#     encoded_positive = Add(name = 'merge_features_pos')([bug_i_pos_feat_mlp, bug_t_pos_feat_lstm, bug_d_pos_feat_cnn])
#     encoded_negative = Add(name = 'merge_features_neg')([bug_i_neg_feat_mlp, bug_t_neg_feat_lstm, bug_d_neg_feat_cnn])
    
    encoded_anchor = concatenate([bug_i_in_feat_mlp, bug_t_in_feat_lstm, bug_d_in_feat_cnn], name = 'merge_features_in')
    encoded_positive = concatenate([bug_i_pos_feat_mlp, bug_t_pos_feat_lstm, bug_d_pos_feat_cnn], name = 'merge_features_pos')
    encoded_negative = concatenate([bug_i_neg_feat_mlp, bug_t_neg_feat_lstm, bug_d_neg_feat_cnn], name = 'merge_features_neg')
    
    # Reduction
#     encoded_anchor = Dense(100, activation='relu')(encoded_anchor)
#     encoded_positive = Dense(100, activation='relu')(encoded_positive)
#     encoded_negative = Dense(100, activation='relu')(encoded_negative)
    
    # Normalization
    encoded_anchor = Lambda(normalize, name='normalize_encoded_anchor')(encoded_anchor)
    encoded_positive = Lambda(normalize, name='normalize_encoded_pos')(encoded_positive)
    encoded_negative = Lambda(normalize, name='normalize_encoded_neg')(encoded_negative)

    positive_d = Lambda(cosine_distance, name='pos_cosine_distance', output_shape=[1])([encoded_anchor, encoded_positive])
    negative_d = Lambda(cosine_distance, name='neg_cosine_distance', output_shape=[1])([encoded_anchor, encoded_negative])
    
    # Loss function only works with a single output
    output = Lambda(
        lambda vects: stack_tensors(vects),
        name='stack-distances',
        output_shape=(2, 1)
    )([positive_d, negative_d])
  
    similarity_model = Model(inputs = [bug_t_in, bug_t_pos, bug_t_neg, 
                                       bug_d_in, bug_d_pos, bug_d_neg, 
                                       bug_i_in, bug_i_pos, bug_i_neg], 
                           outputs = output, name = 'Similarity_Model')
    
    # setup the optimization process 
    similarity_model.compile(optimizer='adam', loss=margin_loss, metrics=[pos_distance, neg_distance])

    return similarity_model

## Experiment

##### Logs

In [23]:
tbCallBack = keras.callbacks.TensorBoard(log_dir=LOG_DIR, histogram_freq=0, write_graph=True, write_images=True)

##### Checkpoint

In [24]:
from keras.callbacks import ModelCheckpoint

def checkpoint_model(name):
    m_dir = os.path.join('checkpoint')
    if not os.path.exists(m_dir):
        os.mkdir(m_dir)
    return ModelCheckpoint(os.path.join(m_dir, "{}.hdf5".format(name)), monitor='loss', \
                                        verbose=1, save_best_only=False, mode='min', period=1)

# checkpoint
checkpoint = checkpoint_model(FILE_PATH)

In [ ]:
%%time
import keras

# Inspired on https://'pastebin.com/TaGFdcBA
# TODO: https://stackoverflow.com/questions/49941903/keras-compute-cosine-distance-between-two-flattened-outputs
keras.backend.clear_session()

number_of_columns_info = valid_input_sample['info'].shape[1]

cnn_feature_model = cnn_model(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.word_index) + 1, 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_D, 
                              trainable=False)

lstm_feature_model = lstm_model(embeddings=baseline.embedding_matrix, 
                              num_words=len(baseline.word_index) + 1, 
                              embedding_dim=EMBEDDING_DIM, 
                              max_sequence_length=MAX_SEQUENCE_LENGTH_T, 
                              trainable=False)

mlp_feature_model = mlp_model(number_of_columns_info)

similarity_model = siamese_model(lstm_feature_model, cnn_feature_model, mlp_feature_model, 
                                     number_of_columns_info, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

# cnn_feature_model.summary()
# lstm_feature_model.summary()
similarity_model.summary()

'''
    Experiments log
    
    
'''
h = similarity_model.fit_generator(train_gen, 
                               steps_per_epoch = 512,
                                             epochs = 20,
                                             verbose = 1,
                                             validation_data=test_gen,
                                               # callbacks=[tbCallBack, checkpoint]
                                              )  # 

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
info_in (InputLayer)            (None, 1682)         0                                            
__________________________________________________________________________________________________
title_in (InputLayer)           (None, 20)           0                                            
__________________________________________________________________________________________________
desc_in (InputLayer)            (None, 200)          0                                            
__________________________________________________________________________________________________
info_pos (InputLayer)           (None, 1682)         0                                            
__________________________________________________________________________________________________
title_pos 

307/512 [================>.............] - ETA: 37s - loss: 0.9086 - pos_distance: 0.5441 - neg_distance: 0.4527

In [ ]:
loss=h.history['loss']
val_loss=h.history['val_loss']

plt.plot(loss, label='loss')
plt.plot(val_loss, label='val_loss')
plt.title('Model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

In [27]:
def save_model(model, name):
    m_dir = os.path.join('modelos')
    if not os.path.exists(m_dir):
        os.mkdir(m_dir)
    model.save(os.path.join(m_dir, "model_{}.h5".format(name)))
    print("Saved model to disk")

In [28]:
# save_model(similarity_model, SAVE_PATH)

Saved model to disk


### Using the feature layers

In [29]:
def get_info(baseline, bug):
    info = np.concatenate((
        baseline.to_one_hot(bug['bug_severity'], baseline.info_dict['bug_severity']),
        baseline.to_one_hot(bug['bug_status'], baseline.info_dict['bug_status']),
        baseline.to_one_hot(bug['component'], baseline.info_dict['component']),
        baseline.to_one_hot(bug['priority'], baseline.info_dict['priority']),
        baseline.to_one_hot(bug['product'], baseline.info_dict['product']),
        baseline.to_one_hot(bug['version'], baseline.info_dict['version']))
    )
    return info

#### Similarity cosine 

In [30]:
def cosine_normalized(a, b):
    a = K.variable(a)
    b = K.variable(b)
    # normalization
    a = normalize(a)
    b = normalize(b)
    # reshape
    a = K.reshape(a, (a.shape[0], 1))
    b = K.reshape(b, (1, b.shape[0]))
    # dot
    cos_sim = K.batch_dot(a, b, axes=1)
    return K.eval(K.mean(K.constant(1) - cos_sim))

def cosine(a, b):
    a = K.variable(a)
    b = K.variable(b)
    # reshape
    a = K.reshape(a, (a.shape[0], 1))
    b = K.reshape(b, (1, b.shape[0]))
    # dot
    cos_sim = K.batch_dot(a, b, axes=1)
    return K.eval(K.mean(K.constant(1) - cos_sim))
#     return 1 - spatial.distance.cosine(a, b)

#### Loading bugs of test

In [595]:
from scipy import spatial
# bug_id = [96204, 2] # non-duplicate {15196, 2}
bug_id = [96204, 85581] # duplicate {85581, 96204, 106979}
bug_set = baseline.get_bug_set()
dup_a, dup_b = bug_id
bug_a = bug_set[dup_a]
bug_b = bug_set[dup_b]

#### LSTM feature

In [596]:
bug_a['title'], bug_b['title']

('preferences person text cut off using default fonts on organization',
 'preferences the type filter text is cut off in the preference dialog')

In [ ]:
bug_vector_a_t = lstm_feature_model.predict(np.array([bug_a['title_word']]))[0]
bug_vector_b_t = lstm_feature_model.predict(np.array([bug_b['title_word']]))[0]
result = cosine_normalized(bug_vector_a_t, bug_vector_b_t)
result

In [ ]:
bug_vector_a_t, bug_vector_b_t

#### CNN feature

In [ ]:
bug_a['description'], bug_b['description']

In [ ]:
bug_vector_a_d = cnn_feature_model.predict(np.array([bug_a['description_word']]))[0]
bug_vector_b_d = cnn_feature_model.predict(np.array([bug_b['description_word']]))[0]
result = cosine_normalized(bug_vector_a_d, bug_vector_b_d)
result

In [ ]:
bug_vector_a_d, bug_vector_b_d

#### MLP feature

In [ ]:
bug_vector_a_i = mlp_feature_model.predict(np.array([get_info(baseline, bug_a)]))[0]
bug_vector_b_i = mlp_feature_model.predict(np.array([get_info(baseline, bug_b)]))[0]
result = cosine_normalized(bug_vector_a_i, bug_vector_b_i)
result

In [ ]:
bug_vector_a_i, bug_vector_b_i

#### Merge features

In [ ]:
bug_vector_a = np.concatenate([ bug_vector_a_i, bug_vector_a_t, bug_vector_a_d ], -1)
bug_vector_b = np.concatenate([ bug_vector_b_i, bug_vector_b_t, bug_vector_b_d ], -1)
result = cosine_normalized(bug_vector_a, bug_vector_b)
result

In [ ]:
bug_vector_a, bug_vector_b

#### Using the siamese model

In [399]:
bug_title =  similarity_model.get_layer('title_in').input # Input(shape = (MAX_SEQUENCE_LENGTH_T, ), name = 'title')
bug_desc =  similarity_model.get_layer('desc_in').input # Input(shape = (MAX_SEQUENCE_LENGTH_D, ), name = 'desc')
bug_info = similarity_model.get_layer('info_in').input # Input(shape = (MAX_SEQUENCE_LENGTH_I, ), name = 'info') # 

title_encoder = similarity_model.get_layer('FeatureLstmGenerationModel')
desc_encoder = similarity_model.get_layer('FeatureCNNGenerationModel')
info_encoder = similarity_model.get_layer('FeatureMlpGenerationModel')

bug_t = title_encoder(bug_title)
bug_d = desc_encoder(bug_desc)
bug_i = info_encoder(bug_info)
# Representation layer
model = similarity_model.get_layer('merge_features_in')
output = model([bug_i, bug_t, bug_d])
# normalization = similarity_model.get_layer('normalize_encoded_anchor')
# output = normalization(output)

model = Model(inputs=[bug_title, bug_desc, bug_info], outputs=[output])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [400]:
bug_vector_a = model.predict([ [bug_a['title_word']], [bug_a['description_word']], [get_info(baseline, bug_a)] ])[0]
bug_vector_b = model.predict([ [bug_b['title_word']], [bug_b['description_word']], [get_info(baseline, bug_b)] ])[0]
result = cosine_normalized(bug_vector_a, bug_vector_b)
result

0.9317614

### Retrieval evaluation

In [45]:
from methods.retrieval import Retrieval
from annoy import AnnoyIndex
import numpy as np

In [46]:
retrieval = Retrieval()

path = 'data/processed/{}'.format(DOMAIN)
path_buckets = 'data/normalized/{}/{}.csv'.format(DOMAIN, DOMAIN)
path_train = 'data/processed/{}/train.txt'.format(DOMAIN)
path_test = 'data/processed/{}/test.txt'.format(DOMAIN)

MAX_SEQUENCE_LENGTH_T = 20 # Title
MAX_SEQUENCE_LENGTH_D = 200 # Description
MAX_SEQUENCE_LENGTH_I = 1682 # Status, Severity, Version, Component, Module

# Create the instance from baseline
retrieval.baseline = Baseline(path, path_buckets, MAX_SEQUENCE_LENGTH_T, MAX_SEQUENCE_LENGTH_D)

df = pd.read_csv(path_buckets)

# Load bug ids
retrieval.load_bugs(path, path_train)
# Create the buckets
retrieval.create_bucket(df)
# Read and create the test queries duplicate
retrieval.create_queries(path_test)

Reading train data
Reading the test...



Reading test data



Creating the buckets...



Creating the queries...


In [47]:
issues_by_buckets = {}
for bucket in tqdm(retrieval.buckets):
    issues_by_buckets[bucket] = bucket
    for issue in np.array(retrieval.buckets[bucket]).tolist():
        issues_by_buckets[issue] = bucket

In [48]:
retrieval.train_vectorized, retrieval.test_result = [], []
# Infer vector to all train
retrieval.read_train(path_train)

In [49]:
print("Total of queries:", len(retrieval.test))

Total of queries: 12659


#### Selecting bugs from test

In [50]:
buckets_train = set()
for row in tqdm(retrieval.test):
    bug_id, ground_truth = row
    vectorizer = [bug_id] 
    vectorizer += ground_truth
    for test_bug_id in vectorizer:
        buckets_train.add(issues_by_buckets[test_bug_id])

#### Model to vectorize

In [ ]:
import keras
from keras.models import Model
from keras.layers import Input
from keras.models import load_model

bug_title =  similarity_model.get_layer('title_in').input 
bug_desc =  similarity_model.get_layer('desc_in').input 
bug_info = similarity_model.get_layer('info_in').input 

title_encoder = similarity_model.get_layer('FeatureLstmGenerationModel')
desc_encoder = similarity_model.get_layer('FeatureCNNGenerationModel')
info_encoder = similarity_model.get_layer('FeatureMlpGenerationModel')

bug_t = title_encoder(bug_title)
bug_d = desc_encoder(bug_desc)
bug_i = info_encoder(bug_info)
# Representation layer
model = similarity_model.get_layer('merge_features_in')
output = model([bug_i, bug_t, bug_d])
# Normalization
# model_normalized = similarity_model.get_layer('normalize_encoded_anchor')
# output = model_normalized(output)

model = Model(inputs=[bug_title, bug_desc, bug_info], outputs=[output])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

In [ ]:
model.summary()

#### Vectorizing bugs from train

In [ ]:
def vectorizer_buckets_train(buckets_train):
    bug_set = retrieval.baseline.get_bug_set()
    buckets_train_vectorized = []
    for bug_id in tqdm(buckets_train): # retrieval.bugs_train
        bug = bug_set[bug_id]
        bug_vector = model.predict([ [bug['title_word']], [bug['description_word']], [retrieval.get_info(bug)] ])[0]
        buckets_train_vectorized.append({ 'bug_id' : bug_id, 'vector' : bug_vector })
    return buckets_train_vectorized

In [ ]:
buckets_train_vectorized = vectorizer_buckets_train(buckets_train)

#### Vectorizing bugs from test

In [ ]:
bug_set = retrieval.baseline.get_bug_set()
queries_test_vectorized = []
for row in tqdm(retrieval.test):
    bug_id, ground_truth = row
    vectorizer = [bug_id] 
    vectorizer += ground_truth
    for test_bug_id in vectorizer:
        if issues_by_buckets[test_bug_id] == test_bug_id: continue # if the bug is the master
        bug = bug_set[test_bug_id]
        bug_vector = model.predict([ [bug['title_word']], [bug['description_word']], [retrieval.get_info(bug)] ])[0]
        queries_test_vectorized.append({ 'bug_id' : test_bug_id, 'vector' : bug_vector,
                                        'ground_truth': issues_by_buckets[test_bug_id] })

#### Indexing bugs

In [ ]:
# Indexing all train
def indexing_train(buckets_train_vectorized):
    X = np.array(buckets_train_vectorized)
    annoy = AnnoyIndex(X[0]['vector'].shape[0])  # Length of item vector that will be indexed

    loop = tqdm(total=len(X))
    for index, row in enumerate(X):
        vector = row['vector']
        annoy.add_item(index, vector)
        loop.update(1)
    loop.close()
    annoy.build(10) # 10 trees
    return annoy

In [ ]:
annoy = indexing_train(buckets_train_vectorized)

#### Getting the list of candidates

In [ ]:
def indexing_test(queries_test_vectorized):
    X_test = queries_test_vectorized
    distance_test, indices_test = [], []
    for index, row in tqdm(enumerate(X_test)):
        vector = row['vector']
        rank, dist = annoy.get_nns_by_vector(vector, 30, include_distances=True)
        indices_test.append(rank)
        distance_test.append(1 - np.array(dist)) # normalize the similarity between 0 and 1
    return X_test, distance_test, indices_test

In [ ]:
X_test, distance_test, indices_test = indexing_test(queries_test_vectorized)

In [ ]:
print("Total buckets train vectorized: {}".format(len(buckets_train_vectorized)))
print("Total queries vectorized: {}".format(len(queries_test_vectorized)))

### Rank result

In [ ]:
formated_rank = []
for row_index, row_sim in tqdm(zip(indices_test, distance_test)):
    row_index, row_sim = row_index[:20], row_sim[:20]
    formated_rank.append(",".join(["{}:{}".format(buckets_train_vectorized[index]['bug_id'], sim) 
                                   for index, sim in zip(row_index, row_sim)]))

#### Queries

In [ ]:
# Generating the rank result
rank_queries = []

for index, row in tqdm(enumerate(X_test)):
    dup_a, ground_truth = row['bug_id'], row['ground_truth']
    rank_queries.append("{}:{}".format(dup_a, ground_truth))

In [ ]:
exported_rank = []
loop = tqdm(total=len(rank_queries))

for query, rank in zip(rank_queries, formated_rank):
    exported_rank.append("{}|{}".format(query, rank))
    loop.update(1)
loop.close()

In [ ]:
exported_rank[:20]

In [ ]:
with open(os.path.join(path, 'exported_rank.txt'), 'w') as file_out:
    for row in exported_rank:
        file_out.write(row + "\n")

In [ ]:
class Evaluation():
    def __init__(self):
        pass
    
    """
        Rank recall_rate_@k
        rank = "query:master|master:id:sim,master:id:sim"
    """
    def top_k_recall(self, rank, k):
        query, rank = rank.split('|')
        query_dup_id, query_master = query.split(":")
        query_master = int(query_master)
        rank_masters = [int(item.split(':')[0]) for pos, item in enumerate(rank.split(",")[:20])]
        corrects = len(set([query_master]) & set(rank_masters[:k]))
        #total = len(retrieval.buckets[issues_by_buckets[query_master]])
        total = 1 if corrects <= 0 else corrects
        return float(corrects), total

    def evaluate(self, path):
        recall_at_5_corrects_sum, recall_at_10_corrects_sum, recall_at_15_corrects_sum, recall_at_20_corrects_sum = 0, 0, 0, 0
        recall_at_5_total_sum, recall_at_10_total_sum, recall_at_15_total_sum, recall_at_20_total_sum = 0, 0, 0, 0
        print("Evaluating...")
        with open(path, 'r') as file_input:
            for row in file_input:
                #if row == '': continue
                recall_at_5_corrects, recall_at_5_total = self.top_k_recall(row, k=5)
                recall_at_10_corrects, recall_at_10_total = self.top_k_recall(row, k=10)
                recall_at_15_corrects, recall_at_15_total = self.top_k_recall(row, k=15)
                recall_at_20_corrects, recall_at_20_total = self.top_k_recall(row, k=20)
                
                recall_at_5_corrects_sum += recall_at_5_corrects
                recall_at_10_corrects_sum += recall_at_10_corrects
                recall_at_15_corrects_sum += recall_at_15_corrects
                recall_at_20_corrects_sum += recall_at_20_corrects
                recall_at_5_total_sum += recall_at_5_total
                recall_at_10_total_sum += recall_at_10_total
                recall_at_15_total_sum += recall_at_15_total
                recall_at_20_total_sum += recall_at_20_total
        
        report = {
            '1 - recall_at_5' : round(recall_at_5_corrects_sum / recall_at_5_total_sum, 2),
            '2 - recall_at_10' : round(recall_at_10_corrects_sum / recall_at_10_total_sum, 2),
            '3 - recall_at_15' : round(recall_at_15_corrects_sum / recall_at_15_total_sum, 2),
            '4 - recall_at_20' : round(recall_at_20_corrects_sum / recall_at_20_total_sum, 2)
        }

        return report

In [ ]:
evaluation = Evaluation()
report = evaluation.evaluate(os.path.join(path, 'exported_rank.txt'))
report